In [2]:
import os
import shutil
from glob import glob

# Define source directory
source_dir = '/root/speechbrain/templates/speaker_id/results/speaker_id/1986/save'

# Copy label encoder to the current directory
shutil.copy(os.path.join(source_dir, 'label_encoder.txt'), os.getcwd())

# Copy the latest checkpoint to the current directory
latest_checkpoint = max(glob(os.path.join(source_dir, 'CKPT*')), key=os.path.getctime)
for file in os.listdir(latest_checkpoint):
    file_source = os.path.join(latest_checkpoint, file)
    shutil.copy(file_source, os.getcwd())


In [6]:
from speechbrain.pretrained import EncoderClassifier
import sys

source="/root/inference/"
savedir="/root/inference/"

sys.path.append(source)

import hparams_inference as hi

pretrainer = hi.pretrainer
pretrainer.set_collect_in(savedir)

from speechbrain.utils.distributed import run_on_main
run_on_main(
    pretrainer.collect_files, kwargs={"default_source": source},
)
pretrainer.load_collected(device="cpu")

# Now return the system
classifier = EncoderClassifier(hi.modules, hi.hparams, {})

In [7]:
import torchaudio

# Perform classification
audio_file = '/root/autodl-tmp/LibriSpeech/train-clean-5/5789/70653/5789-70653-0036.flac'
signal, fs = torchaudio.load(audio_file) # test_speaker: 5789
output_probs, score, index, text_lab = classifier.classify_batch(signal)
print('Target: 5789, Predicted: ' + text_lab[0])

# Another speaker
audio_file = '/root/autodl-tmp/LibriSpeech/train-clean-5/460/172359/460-172359-0012.flac'
signal, fs =torchaudio.load(audio_file) # test_speaker: 460
output_probs, score, index, text_lab = classifier.classify_batch(signal)
print('Target: 460, Predicted: ' + text_lab[0])

# And if you want to extract embeddings...
embeddings = classifier.encode_batch(signal)

/root/miniconda3/lib/python3.8/site-packages/speechbrain/dataio/encoder.py:722: UserWarning: CategoricalEncoder.expect_len was never called: assuming category count of 28 to be correct! Sanity check your encoder using `.expect_len`. Ensure that downstream code also uses the correct size. If you are sure this does not apply to you, use `.ignore_len`.
  warnings.warn(


Target: 5789, Predicted: 5789
Target: 460, Predicted: 460
